In [1]:
import numpy as np # linear algerbra
import pandas as pd # data processing

In [2]:
data = pd.read_csv('../ProjectBookRatingPredictions/DataSet/Train_data.csv',error_bad_lines=False)
data.head(5)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,8253,Little Town on the Prairie (Little House #7),Laura Ingalls Wilder/Garth Williams,4.19,0060885432,9780060885434,eng,374,71853,646,1/1/2007,HarperTrophy
1,11101,First Comes Love (Hot Water California #1),Christie Ridgway,3.81,0380818957,9780380818952,en-US,384,1051,47,1/1/2002,Avon Books
2,28193,When Santa Fell to Earth,Cornelia Funke/Paul Howard/Oliver G. Latsch,3.76,043978204X,9780439782043,eng,167,1840,245,10/1/2006,Chicken House / Scholastic
3,25047,The Case of the Stinky Science Project (Jigsaw...,James Preller/John Speirs/James Preller,3.83,0439114284,9780439114288,en-US,80,162,11,3/1/2001,Scholastic Paperbacks
4,17800,Dance Dance Dance,Haruki Murakami,4.04,0099448769,9780099448761,eng,393,50169,2117,2/7/2002,Vintage


In [3]:
data.shape

(10000, 12)

In [4]:
import jieba
def word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [5]:
data["title_cutted"] = data["title"].apply(word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LiuYiNan\AppData\Local\Temp\jieba.cache
Loading model cost 0.493 seconds.
Prefix dict has been built successfully.


In [6]:
#data["title_cutted"] = ''.join(data['title'].str.replace('(','').str.replace(')','').str.split())

In [7]:
data["title_cutted"].head(5)

0    Little   Town   on   the   Prairie     ( Littl...
1    First   Comes   Love   ( Hot   Water     Calif...
2                     When   Santa   Fell   to   Earth
3    The   Case   of   the   Stinky   Science   Pro...
4                                Dance   Dance   Dance
Name: title_cutted, dtype: object

In [8]:
data["title_cutted"] = data["title_cutted"].str.replace('(','').str.replace(')','')

In [9]:
data["title_cutted"].head(5)

0    Little   Town   on   the   Prairie      Little...
1    First   Comes   Love    Hot   Water     Califo...
2                     When   Santa   Fell   to   Earth
3    The   Case   of   the   Stinky   Science   Pro...
4                                Dance   Dance   Dance
Name: title_cutted, dtype: object

## 文本向量化

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(data["title_cutted"])

## using LDA

In [11]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [12]:
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, random_state=0)

In [13]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
    for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [14]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
life great philosophy good mystery shadow introduction easy tolkien summer game century ring island inside times early boys ice paradise
Topic #1:
tales la tale castle shakespeare dreams library adventure novel case mr battle peter ghost les madness princess jones stone scarlet
Topic #2:
vol world war new art short woman long city notes alchemist ii end diary henry iv fullmetal fall 14 work
Topic #3:
guide story american death volume god best harry women essays murder potter heart books high days year valley jane writing
Topic #4:
star like wars poems selected collected big fiction 12 christmas don le theory essential blue 15 literature far discworld anthology
Topic #5:
stories time el 10 journey modern jack saga red cat biography day living code memoir child sword winter song making
Topic #6:
man trilogy dark writings family sea men young works empire old country dance shannara daughter faith laura divine shadows volume
Topic #7:
black adventures 11 power magic science moon 

In [15]:
data["title_cutted"].head(5)

0    Little   Town   on   the   Prairie      Little...
1    First   Comes   Love    Hot   Water     Califo...
2                     When   Santa   Fell   to   Earth
3    The   Case   of   the   Stinky   Science   Pro...
4                                Dance   Dance   Dance
Name: title_cutted, dtype: object

In [16]:
lda.transform(tf)

array([[0.02      , 0.02      , 0.02      , ..., 0.02      , 0.82      ,
        0.02      ],
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.775     ,
        0.025     ],
       [0.05      , 0.05      , 0.05      , ..., 0.05      , 0.05      ,
        0.55      ],
       ...,
       [0.05      , 0.05      , 0.55      , ..., 0.05      , 0.05      ,
        0.05      ],
       [0.03333333, 0.36666706, 0.03333333, ..., 0.03333333, 0.03333333,
        0.03333333],
       [0.03333333, 0.03333333, 0.03333333, ..., 0.03333333, 0.36666688,
        0.03333333]])